In [537]:
#working on parsing the data using pandas.read_csv()

import pandas as pd
import glob


def parseRtergHtml(html):
    colNames1 = ["Eventname", "oDate", "oTime", "Lat.", "Long.", "Depth", "Me", "Txo",
                 "Ehf", "Ebb", "Mehf", "Ehf/Tr^3", "Nstats", "colon", "SRC", "iMag"]
    try:
        df1 = pd.read_csv(html, names = colNames1, skiprows = 1, nrows = 1, delim_whitespace=True)
        df1["SRC"] = df1.iloc[0]["SRC"][5:7]
        iMagTypeVal = df1.iloc[0]["iMag"].split("=")[0]
        df1.insert(14, "iMagType", iMagTypeVal)
        df1["iMag"] = df1.iloc[0]["iMag"].split("=")[1][:-2]
        oTimeVal = df1["oDate"] + " " + df1["oTime"]
        df1["oTime"] = pd.to_datetime(oTimeVal, utc=True)
        intSkip1 = 0
    except:
        df1 = pd.DataFrame(columns = colNames1)
        intSkip1 = 1
    
    del df1["colon"]
    del df1["oDate"]
    
    colNames2 = ["TACER_HF", "TACER_BB"]
    
    try:
        df2 = pd.read_csv(html, names = colNames2, skiprows = 2, nrows = 1, delim_whitespace=True, usecols=[7,10])
        float(df2["TACER_HF"][0]) # test to see if there is a valid number
        intSkip2 = 0
    except:
        df2 = pd.DataFrame(columns = colNames2)
        intSkip2 = 1

    colNames3 = ["junk", "Comment"]
    colNames4 = ["junk", "mTime"]
    colNames5 = ["junk", "iteration"]
    
    df3 = pd.read_csv(html, names = colNames3, skiprows = 3 - intSkip2 - intSkip1, nrows = 1, engine="python", delimiter="S:")
    del df3["junk"]
    
    f = open(html, "r")
    content = f.read().split("\n")
    count = -1
    for lines in content:
        count += 1
    if content[count] == "":
        count -= 1
    
    df4 = pd.read_csv(html, names = colNames4, skiprows = count - 1, nrows = 1, delimiter="|")
    if df4["junk"][0][-5:-1] == "</a>":
        del df4["junk"]
    else:
        del df4["mTime"]
        df4.rename(columns = {"junk" : "mTime"}, inplace=True)
    df4["mTime"] = pd.to_datetime(df4["mTime"], utc=True)
    
    df5 = pd.read_csv(html, names = colNames5, skiprows = count, nrows = 1, delimiter="=")
    del df5["junk"]
    df5["iteration"][0] = df5.iloc[0]["iteration"].split("<")[0]

    df = pd.concat([df1,df2,df3,df4, df5], axis=1)
    return df

In [535]:
def builddf(htmlfiles):
    df = pd.DataFrame()
    for html in htmlfiles:
        try: 
            df1=parseRtergHtml(html)
            if len(df) == 0:  # first run keeps header
                df=df1
            else:  # otherwise strip it
                df=df.append(df1, ignore_index = True)
        except:
            continue
    return df

#htmlfiles = sorted(glob.glob('rterg_html_outs/[0-9]???????.html'))  # creates a sorted version
htmlfiles=glob.glob('rterg_html_outs/bad/*.html')

df = builddf(htmlfiles)
df.head()

C:\Users\Chelsea\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,Eventname,oTime,Lat.,Long.,Depth,Me,Txo,Ehf,Ebb,Mehf,Ehf/Tr^3,Nstats,iMagType,SRC,iMag,TACER_HF,TACER_BB,Comment,mTime,iteration
0,10500,2000-01-05 07:40:40+00:00,-11.27,165.25,15,6.28,83,1.3e+13,5.9e+13,6.31,2.3e+07,19,M_,__,6.0,NaN,NaN,Solomon Islands : IRIS Fetched Data,2016-12-06 22:55:40+00:00,A
1,10800,2000-01-08 01:19:46+00:00,-9.78,159.99,15,6.32,78,1.5e+13,6.8e+13,6.34,3.1e+07,24,M_,__,6.4,NaN,NaN,Solomon Islands : IRIS Fetched Data,2016-12-06 23:28:08+00:00,A
2,30693,1993-03-06 03:05:48+00:00,-10.73,164.02,15,0,538,0,-inf,1,0,0,M_,__,7.1,NaN,NaN,Solomon Islands : IRIS Fetched Data | 1993,2016-12-07 00:13:59+00:00,A
3,51699,1999-05-16 00:51:20+00:00,-4.99,152.76,42,6.93,90,1.9e+14,5.6e+14,7.08,2.6e+08,27,M_,__,7.0,NaN,NaN,Solomon Islands : IRIS Fetched Data | 1999,2016-12-08 17:45:51+00:00,A
4,81695,1995-08-16 10:27:26+00:00,-5.51,153.64,45.6,0,392,0,-inf,4,0,0,M_,__,7.7,NaN,NaN,Solomon Islands : IRIS Fetched Data | 1995,2016-12-08 18:21:34+00:00,A


In [526]:
print(df.index)

RangeIndex(start=0, stop=4, step=1)


In [2]:
# takes in a dataframe and parses it into xml format

import xml.etree.ElementTree as et

def toXML(df):
    
    name = str(df[df.columns[0]][0]) + ".xml"
    
    root = et.Element("root")
    tree = et.ElementTree(root)
    event = et.Element("event")
    root.append(event)
    
    for x in range(20):
        data = str(df[df.columns[x]][0])
        type = et.SubElement(event, df.columns[x]).text = data
    
    file = et.ElementTree(root)
    file.write(name, xml_declaration=True)
    return

toXML(df)

# don't run below here

In [1]:
#builds and empty dataframe

# old testing, do not run.

import pandas as pd

def buildDF():
    colNames = ["#Eventname", "Date", "Otime", "Lat.", "Long.", "Depth", "Me", "Tr", "Ehf", "Ebb",
                "Mehf", "Ehf/Tr^3", "Nstats : Comments", "TACER_HF", "TACER_BB", "COMMENTS"]
    df = pd.DataFrame(columns = colNames)
    return df

#uses a helper function to input data into the dataframe

def parseRtergHtml(df, html):
    data = readHtml(html)
    for x in range(16):
        df[df.columns[x]] = [data[x]]
    return

#creates a list of data from the html file that will be
#inputted into the dataframe

def readHtml(html):
    file = open(html, "r")
    file.readline()
    line2 = file.readline()
    dataList = line2.split()
    str = ""
    for x in range(3):
        str = str + " " + dataList.pop(13)
    dataList[12] = dataList[12] + str
    line3 = file.readline()
    tempList = line3.split()
    dataList.append(tempList[7])
    dataList.append(tempList[10])
    line4 = file.readline()
    dataList.append(line4[15:].strip())
    return dataList